# Cleaning 1.0 - Combine individual excel files into 2 datasets: (1) individual-level dataset, (2) spark q-level dataset, (3) equipment-type-panel-level dataset

In [1]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[0]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import range_boundaries
import os
import shutil
import matplotlib.pyplot as plt
import string
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [2]:
# Load data
labs = pd.read_csv(config.EL_RAW_SAMPLE / "final_sample_with_EL_file_status.csv")

# BL surveys folder
bl_surveys_folder = config.BL_RAW_SURVEY / "1_LabExcels"

# EL surveys folder
el_surveys_folder = config.EL_RAW_SURVEY / "1_LabExcels"

In [3]:
# Load survey dictionaries (helper for mapping variables to survey files, sheets and cells)
other_qs_dict = pd.read_excel(config.SURVEY_DICTIONARIES / "helper_survey_dictionary.xlsx", sheet_name="Other")
el_check_rows_dict = pd.read_excel(config.SURVEY_DICTIONARIES / "helper_survey_dictionary.xlsx", sheet_name="EL check")
spec_equip_cols_dict = pd.read_excel(config.SURVEY_DICTIONARIES / "helper_survey_dictionary.xlsx", sheet_name="Specialized equipment")

## Individual-level dataset: 
- survey dates (BL + EL)
- characteristics (BL)
- sharing and comms (BL)
- consent to merge (BL)
- awareness (EL)
- attitudes (EL)
- calculator (calc, T only)
- waste (BL)
- specialized equipment (BL)
- checks for changes (EL)

In [4]:
# List of labgroupids to process
labgroupids = labs['labgroupid'].unique().tolist()

labs = labs.copy()

In [5]:
# Function to create variable(s) from cell(s) in individual survey files
def create_var(ws, labs, mask, var_name,
               cell,
               multiple_cells=False,
               no_variables=False,
               comment_cell=None,
               fc_cell=None) :

    # Multiple cells
    if multiple_cells:
        cells = ws[cell]  # cell range
        values = [c.value for row in cells for c in row]
        values = ["" if v is None else v for v in values]

        if comment_cell:
            comment_cells = ws[comment_cell] # comment range
            comment_values = [c.value for row in comment_cells for c in row]
            comment_values = ["" if v is None else v for v in comment_values]

        if fc_cell:
            fc_cells = ws[fc_cell] # free text range
            fc_values = [c.value for row in fc_cells for c in row]
            fc_values = ["" if v is None else v for v in fc_values]

        # Create separate variables per cell (cell, comment, free text)
        if no_variables: 
            for i, v in enumerate(values, start=1):
                labs.loc[mask, f"{var_name}_{i}"] = v

            if comment_cell:
                for i, v in enumerate(comment_values, start=1):
                    labs.loc[mask, f"{var_name}_{i}_co"] = v
            
            if fc_cell:
                for i, v in enumerate(fc_values, start=1):
                    labs.loc[mask, f"{var_name}_{i}_fc"] = v

        # Create single variable with all cells joined by ";"
        else:
            labs.loc[mask, var_name] = ";".join(str(v) for v in values if v not in (None, ""))

            if comment_cell:
                labs.loc[mask, f"{var_name}_co"] = ";".join(str(v) for v in comment_values if v not in (None, ""))
            
            if fc_cell:
                labs.loc[mask, f"{var_name}_fc"] = ";".join(str(v) for v in fc_values if v not in (None, ""))

    # Single cell
    else:
        v = ws[cell].value
        labs.loc[mask, var_name] = "" if v is None else v

        # Comment
        if comment_cell:
            v = ws[comment_cell].value
            labs.loc[mask, f"{var_name}_co"] = "" if v is None else v


        # Free text response
        if fc_cell:
            v = ws[fc_cell].value
            labs.loc[mask, f"{var_name}_fc"] = "" if v is None else v


In [6]:
for labgroupid in labgroupids:

    mask = labs["labgroupid"] == labgroupid
    if not mask.any():
        continue

    treatment_status = labs.loc[mask, "Treatment Status"].iloc[0]

    bl_path = bl_surveys_folder / f"BL_{labgroupid}.xlsx"
    if not bl_path.exists():
        continue

    wb = load_workbook(bl_path, data_only=True)

    for _, row in other_qs_dict.iterrows():

        if row["Survey"] != "BL":
            continue

        tc_only = row["T or C only"]
        if pd.notna(tc_only) and tc_only != treatment_status:
            continue

        ws = wb[row["Sheet"]]

        create_var(
            ws=ws,
            labs=labs,
            mask=mask,
            var_name=row["Variable"],
            cell=row["Cell(s)"],
            multiple_cells=(row["Multiple cells"] == "Y"),
            no_variables=pd.notna(row["No variables"]),
            comment_cell=row["Comment"] if pd.notna(row["Comment"]) else None,
            fc_cell=row["Free text"] if pd.notna(row["Free text"]) else None
        )


/var/folders/nn/21zflm3n7gzc5spw42wq352rpm87xt/T/ipykernel_74812/1927288284.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  labs.loc[mask, var_name] = "" if v is None else v
/var/folders/nn/21zflm3n7gzc5spw42wq352rpm87xt/T/ipykernel_74812/1927288284.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  labs.loc[mask, var_name] = "" if v is None else v
/var/folders/nn/21zflm3n7gzc5spw42wq352rpm87xt/T/ipykernel_74812/1927288284.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible 

## Equipment-type-panel-level dataset:
- survey: "bl" or "el"
- equipment
- type
- equipment-specific fields

In [ ]:
# Equipment sheets to check and their ranges (survey sheet name, calc sheet name)
equipment_sheet_mappings = {
    "fumecupboards": {"survey_sheet": "2. Fume Cupboards", "calc_sheet": "Fume Cupboards"},
    "fridges": {"survey_sheet": "3. Fridges (4°C)", "calc_sheet": "Fridges (4°C)"},
    "freezers": {"survey_sheet": "4. Freezers (-20°C)", "calc_sheet": "Freezers (-20°C)"},
    "ults": {"survey_sheet": "5. ULT Freezers", "calc_sheet": "ULT Freezers"},
    "glassware": {"survey_sheet": "6. Glassware Drying Cabinets", "calc_sheet": "Glassware Drying Cabinets"},
    "microbio": {"survey_sheet": "7. Microbio Safety Cabinets", "calc_sheet": "Microbio Safety Cabinets"},
    "cryostats": {"survey_sheet": "8. Cryostats", "calc_sheet": "Cryostats"},
    "baths": {"survey_sheet": "9. Water Baths", "calc_sheet": "Water Baths"},
    "incubators": {"survey_sheet": "10. CO2 Incubators", "calc_sheet": "CO2 Incubators"},
    "heaters": {"survey_sheet": "11. Block Heaters", "calc_sheet": "Block Heaters"},
    "it": {"survey_sheet": "12. IT Equipment", "calc_sheet": "IT Equipment"},
}

In [4]:
# Function to check whether each sheet contains any data
def has_data(ws, cell_range):
    min_col, min_row, max_col, max_row = range_boundaries(cell_range)
    for row in ws.iter_rows(min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col, values_only=True):
        if any(cell is not None and str(cell).strip() != "" for cell in row):
            return True
    return False

In [ ]:
# Define the fields for each equipment type
equipment_fields = {
    # Fume cupboards
    "fumecupboards": [
        ("controller_type", 6),
        ("sash_width_mc", 7),
        ("sash_width_fc", 8),
        ("face_velocity_mc", 9),
        ("face_velocity_fc", 10),
        ("no", 11),
        ("lifted", 12),
        ("hours", 13),
        ("days_mc", 14),
        ("days_fc", 15),
        ("surface", 16),
        ("sharing", 17),
        ("freq_el_check", 18)
    ],
    # Fridges
    "fridges": [
        ("size", 6),
        ("door_openings", 7),
        ("no", 8),
        ("sharing", 9),
        ("door_openings_el_check", 10)
    ],
    # Freezers
    "freezers": [
        ("size", 6),
        ("refrigerant", 7),
        ("no", 8),
        ("temp", 9),
        ("icing", 10),
        ("drawers", 11),
        ("door_openings", 12),
        ("sharing", 13),
        ("door_openings_el_check", 14)
    ],
    # ULTs
    "ults": [
        ("ult_type", 6),
        ("size", 7),
        ("temp", 8),
        ("no", 9),
        ("seals", 10),
        ("spacing", 11),
        ("filter", 12),
        ("door_openings", 13),
        ("sharing", 14),
        ("temp_el_check", 15)
    ],
    # Glassware drying cabinets
    "glassware": [
        ("capacity", 6),
        ("tech", 7),
        ("fan", 8),
        ("temp", 9),
        ("no", 10),
        ("days_mc", 11),
        ("days_fc", 12),
        ("hours_mc", 13),
        ("hours_fc", 14),
        ("sharing", 15),
        ("freq_el_check", 16)
    ],
    # Microbio safety cabinets
    "microbio": [
        ("width", 6),
        ("age", 7),
        ("ducting", 8),
        ("no", 9),
        ("days_mc", 10),
        ("days_fc", 11),
        ("hours_mc", 12),
        ("hours_fc", 13),
        ("sharing", 14),
        ("freq_el_check", 15)
    ],
    # Cryostats
    "cryostats": [
        ("temp", 6),
        ("sleep_mode", 7),
        ("no", 8),
        ("days_mc", 9),
        ("days_fc", 10),
        ("hours_mc", 11),
        ("hours_fc", 12),
        ("sharing", 13),
        ("sleep_mode_el_check", 14)
    ],
    # Water baths
    "baths": [
        ("capacity", 6),
        ("heating", 7),
        ("temp_mc", 8),
        ("temp_fc", 9),
        ("no", 10),
        ("lid", 11),
        ("days_mc", 12),
        ("days_fc", 13),
        ("hours_mc", 14),
        ("hours_fc", 15),
        ("sharing", 16),
        ("freq_el_check", 17)
    ],
    # CO2 incubators
    "incubators": [
        ("capacity", 6),
        ("age", 7),
        ("no", 8),
        ("days_mc", 9),
        ("days_fc", 10),
        ("sharing", 11),
        ("freq_el_check", 12)
    ],
    # Block heaters
    "heaters": [
        ("blocks", 6),
        ("temp_mc", 7),
        ("temp_fc", 8),
        ("no", 9),
        ("days_mc", 10),
        ("days_fc", 11),
        ("hours_mc", 12),
        ("hours_fc", 13),
        ("sharing", 14),
        ("freq_el_check", 15)
    ],
    # IT equipment
    "it": [
        ("it_type", 6),
        ("no", 7),
        ("days_mc", 8),
        ("days_fc", 9),
        ("hours_mc", 10),
        ("hours_fc", 11),
        ("monitor", 12),
        ("screens", 13),
        ("sharing", 14),
        ("freq_el_check", 15)
    ],
}

In [16]:
# Function to get value and comment columns for a type (e.g. 1 = C, D)
def get_type_columns(type_number):
    col_index = 2 + (type_number - 1) * 2  # C is index 2
    value_col = string.ascii_uppercase[col_index]
    co_col = string.ascii_uppercase[col_index + 1]
    return value_col, co_col

In [ ]:
# Function to extract data from each equipment type
def extract_equipment_data(labgroupid_list, equipment_key, equipment_sheet_mappings, equipment_fields, max_types=10):
    records = []
    mapping = equipment_sheet_mappings[equipment_key]

    for labgroupid in labgroupid_list:
        try:
            filename = config.BL_RAW_SURVEY / "1_LabExcels" / f"BL_{labgroupid}.xlsx"
            wb = load_workbook(filename, data_only=True, read_only=True)
            ws = wb[mapping["orig_sheet"]]
        except FileNotFoundError:
            print(f"File not found for lab: {labgroupid}")
            continue  # skip this lab
        except KeyError:
            print(f"Sheet not found for lab: {labgroupid} and equipment {mapping['survey_sheet']}")
            continue  # skip this lab
        except Exception as e:
            print(f"Unexpected error opening file for {labgroupid}: {e}")
            continue

        # Loop through all types
        for type_number in range(1, max_types + 1):
            value_col, co_col = get_type_columns(type_number)
            type_has_data = False

            # Check if this type has any data
            for field_name, row_num in equipment_fields[equipment_key]:
                if ws[f"{value_col}{row_num}"].value is not None or ws[f"{co_col}{row_num}"].value is not None:
                    type_has_data = True
                    break
            if not type_has_data:
                continue  # Skip empty types

            # Create one row per lab × type
            row_result = {
                "labgroupid": labgroupid,
                "equipment": equipment_key,
                "type": type_number
            }

            for field_name, row_num in equipment_fields[equipment_key]:
                try:
                    row_result[field_name] = ws[f"{value_col}{row_num}"].value
                    row_result[field_name + "_co"] = ws[f"{co_col}{row_num}"].value
                except Exception as e:
                    print(f"Error reading {value_col}{row_num}/{co_col}{row_num} in lab {labgroupid}: {e}")
                    row_result[field_name] = None
                    row_result[field_name + "_co"] = None

            # Store the row
            records.append(row_result)

    return pd.DataFrame(records)

In [18]:
# Dictionary to hold labgroupids per equipment for treated labs
treated_labs_dict = {}

for equipment_key in sheet_mappings.keys():
    # Boolean mask: only labs that are treatment labs AND have this equipment
    mask = (
        (checked_surveys[equipment_key] == True) &
        (checked_surveys["Treatment Status"] == "treatment")
    )
    # List of labgroupids
    treated_labs = checked_surveys.loc[mask, "labgroupid"].tolist()
    treated_labs_dict[equipment_key] = treated_labs


In [ ]:
# For treated labs, extract data for each equipment type and save to CSV (one row per labgroupid-type)
all_data = {}

for equipment_key, labgroup_list in treated_labs_dict.items():
    if labgroup_list:  # only process if there are labs
        df = extract_equipment_data(
            labgroupid_list=labgroup_list,
            equipment_key=equipment_key,
            equipment_sheet_mappings=equipment_sheet_mappings,
            equipment_fields=equipment_fields,
            max_types=10 
        )
        all_data[equipment_key] = df
        # Save CSV
        df.to_csv(
            config.DATA_ROOT / "13_Calculators" / "3_Reports" / f"{equipment_key}_treated_labs_tidy.csv",
            index=False
        )
